This notebook we discuss quantum classification algorithms implemented in real life. Code examples are presented for different algorithms such as quantum classifiers, support vector machines, and sparse support vector machines.

Quantum machine learning methods are based on supervised and unsupervised learning. Amplitude modification methods are used in the k-nearest neighbor and clustering quantum algorithms.Quantum matrix computations are used for quantum kernel methods. Quantum kernel methods support vector machines and Gaussian processes. Supervised quantum machine learning is the basis for variational quantum classifier methods. [1]

**Quantum Classifier**

In [ ]:
!pip install qclassify

In [ ]:
# Quantum Classifier

import qclassify

from qclassify.preprocessing import *
from qclassify.encoding_circ import *
from qclassify.qclassifier import QClassifier as QuantumClassifier
from qclassify.proc_circ import *
from qclassify.postprocessing import *

encoding_opts={
                'preprocessing':id_func,        
                'encoding_circ':x_product,      
        }





process_opts={
                'proc_circ':layer_xz,   
                'postprocessing':{
                        'quantum':measure_top, 
                        'classical':prob_one, 
                }
        }






quantum_bits = [0, 1] 
classify_opts = {
                'encoder_options':encoding_opts,
                'proc_options':process_opts,
        }
qCircuit = QuantumClassifier(quantum_bits, classify_opts)





circuit_values = [1, 1] 
params = [2.0672044712460114,          1.3311348339721203] 

quantum_circuit_prog = qCircuit.circuit(circuit_values, params) 

print(quantum_circuit_prog)

RX(1) 0
RX(1) 1
CZ 0 1
RX(2.0672044712460114) 0
RX(1.3311348339721203) 1
DECLARE ro BIT[1]
MEASURE 0 ro[0]



**Variational Quantum Classifier**

Variational quantum circuits are used in machine learning. In these circuits, gate parameters are trained using learning methods. The input vectors are the features of the training model. They are equivalent to the quantum system amplitudes. The training method is based on the quantum methods. The classifying solution iterates over the data set, and accuracy is achieved when using the quantum machine learning model and data. The variational quantum classifier algorithm is based on supervised learning. We can use the PennyLane framework for quantum computation and optimization. 
The package has templates based on different variational quantum circuits used for creating and training machine learning models. Embeddings, layers, subroutines, and state preparations are multiple types of quantum circuits. These circuits are called ansatz circuits. [1]

In [ ]:
!pip install pennylane

In [ ]:
# Variational Quantum Classifier

import pennylane as quantumPenny
from pennylane import numpy as nump
from pennylane.optimize import  NesterovMomentumOptimizer as NMO

device = quantumPenny.device('default.qubit', wires=2)


def CalcAngles(xval):

    beta0_val = 2 * nump.arcsin(nump.sqrt(xval[1]) ** 2 / nump.sqrt(xval[0] ** 2 + xval[1] ** 2 + 1e-12) )
    beta1_val = 2 * nump.arcsin(nump.sqrt(xval[3]) ** 2 / nump.sqrt(xval[2] ** 2 + xval[3] ** 2 + 1e-12) )
    beta2_val = 2 * nump.arcsin(nump.sqrt(xval[2] ** 2 + xval[3] ** 2) / nump.sqrt(xval[0] ** 2 + xval[1] ** 2 + xval[2] ** 2 + xval[3] ** 2))

    return nump.array([beta2_val, -beta1_val / 2, beta1_val / 2, -beta0_val / 2, beta0_val / 2])


def CalcState(arr):

    quantumPenny.RY(arr[0], wires=0)

    quantumPenny.CNOT(wires=[0, 1])
    quantumPenny.RY(arr[1], wires=1)
    quantumPenny.CNOT(wires=[0, 1])
    quantumPenny.RY(arr[2], wires=1)

    quantumPenny.PauliX(wires=0)
    quantumPenny.CNOT(wires=[0, 1])
    quantumPenny.RY(arr[3], wires=1)
    quantumPenny.CNOT(wires=[0, 1])
    quantumPenny.RY(arr[4], wires=1)
    quantumPenny.PauliX(wires=0)


def CalcLayer(Warr):

    quantumPenny.Rot(Warr[0, 0], Warr[0, 1], Warr[0, 2], wires=0)
    quantumPenny.Rot(Warr[1, 0], Warr[1, 1], Warr[1, 2], wires=1)

    quantumPenny.CNOT(wires=[0, 1])
    

@quantumPenny.qnode(device)
def FindCircuit(weight, angle=None):

    CalcState(angle)
    
    for W in weight:
        CalcLayer(W)

    return quantumPenny.expval(quantumPenny.PauliZ(0))


def GetVariationalClassifier(var, angle=None):

    weight = var[0]
    bias_val = var[1]

    return FindCircuit(weight, angle=angle) + bias_val


def CalcSquareLoss(label, prediction):
    loss_val = 0
    for l, p in zip(label, prediction):
        loss_val = loss_val + (l - p) ** 2
    loss_val = loss_val / len(label)

    return loss_val


def CalcAccuracy(label, prediction):

    loss_val = 0
    for l, p in zip(label, prediction):
        if abs(l - p) < 1e-5:
            loss_val = loss_val + 1
    loss_val = loss_val / len(label)

    return loss_val


def CalcCost(weight, feature, label):

    prediction = [GetVariationalClassifier(weight, angle=f) for f in feature]

    return CalcSquareLoss(label, prediction)


data_iris_flowers = nump.loadtxt("iris_flower_data.txt")
XVal = data_iris_flowers[:, 0:2]

pad = 0.3 * nump.ones((len(XVal), 1))
XPad = nump.c_[nump.c_[XVal, pad], nump.zeros((len(XVal), 1)) ] 

norm = nump.sqrt(nump.sum(XPad ** 2, -1))
X_norm = (XPad.T / norm).T  

ftrs = nump.array([CalcAngles(x) for x in X_norm])   

YVal = data_iris_flowers[:, -1]

nump.random.seed(0)
num_data = len(YVal)
num_train = int(0.75 * num_data)
index = nump.random.permutation(range(num_data))
feats_train = ftrs[index[:num_train]]
YValT = YVal[index[:num_train]]
ftrs_val = ftrs[index[num_train:]]
Y_val = YVal[index[num_train:]]

quantumbits = 2
layers = 6
varinit = (0.01 * nump.random.randn(layers, quantumbits, 3), 0.0)

opt = NMO(0.01)
batsize = 5

var = varinit
for it in range(50):

    bat_index = nump.random.randint(0, num_train, (batsize, ))
    features_train_batch = feats_train[bat_index]
    Ytrainbat = YValT[bat_index]
    var = opt.step(lambda v: CalcCost(v, features_train_batch, Ytrainbat), var)

    predict_train = [nump.sign(GetVariationalClassifier(var, angle=f)) for f in feats_train]
    predict_val = [nump.sign(GetVariationalClassifier(var, angle=f)) for f in ftrs_val]

    accuracy_train = CalcAccuracy(YValT, predict_train)
    accuracy_val = CalcAccuracy(Y_val, predict_val)

    print("it value: {:5d} | Cost Features: {:0.7f} | Accuracy training: {:0.7f} | Accuracy for validation: {:0.7f} "
          "".format(it+1, CalcCost(var, ftrs, YVal), accuracy_train, accuracy_val))


**Quantum SVM**

The quantum equivalent of SVM uses a feature map to present the quantum circuit information. The kernel concept is used in classification methods. A feature map modified the features. Features are measurable attributes of the subject in context. Classification is about finding the distance between the classes for the input information. The quantum support vector machine is based on a quantum processor for solutions. It is based on supervised learning. Multiclass classifiers are built on quantum support vector machines. A quantum processor has QRAM for handling quantum bits in the memory. [1]

**Quantum Sparse Support Vector Machines** 

There are various techniques such as least squares for simulating quantum sparse support vector machines. The Harrow–Hassidim–Lloyd (HHL) algorithm is used to find a solution for a linear equation–based system to forecast using trained quantum bits. This is a hybrid algorithm of the classical and quantum computing techniques. Regularization is part of the sparse support vector machine The L2 Norm penalty is used as a regularization algorithm, and L1 Norm is better than L2 Norm.

In [ ]:
import renom_q.ml.quantum_svm
quantum_svm = renom_q.ml.quantum_svm.QSVM()
xval = [[3, 4], [5, 6], [7, 9], [8, 6]]
yval = [1, -1, 1, -1]
quantum_svm.plot_graph(xval, yval)

References

[1] Bhagvan Kommadi